# Практика

In [47]:
from lib import AnyName

In [ ]:
from lib import *  # никогда так не делайте

In [ ]:
CONST = 5560

In [ ]:
def useful_func(a, b, c):
    return (a+b+c) * 1.2

In [48]:
class AnyName:
    def __init__(self, grade=3, *args, **kwargs):
        print('Выполняется при обращении к классу')
        # создаются все атрибуты, которые нужны для работы класса
        self.grade = grade
        
        # "приватные" атрибуты
        # если надо запретить изменять атрибут вне кода класса
        self._api_version = 'v1'  # намек на то, что атрибут менять не следует
        self.__secret_attr = 10  # запрет на изменение атрибута вне кода
    
    # функции внутри классов называются методы
    # каждый (почти) метод начинается в переменной self
    def method_1(self):
        # переменная внутри метода ничем не отличается от переменной функции
        x = 1
        
        # self.grade - атрибут - его можно передавать между методами
        self.grade = 1
        print('Секретный атрибут', self.__secret_attr)
        
    def method_2(self):
        self.grade += 1
        
    @staticmethod
    def useful_func(a, b, c):
        # статический метод - просто функция
        return (a+b+c) * 1.2
    
    @property
    def calculated_attr(self):
        # предсказывает значение grade после повышения сотрудника
        if self.grade > 3:
            return self.grade + 2
        else:
            return self.grade + 1

In [49]:
a = AnyName()

Выполняется при обращении к классу


In [50]:
a.grade

3

In [51]:
a.calculated_attr

4

In [46]:
AnyName().useful_func(1, 2, 3)

Выполняется при обращении к классу


7.199999999999999

In [42]:
a.method_1()

Секретный атрибут 10


In [36]:
a._api_version = 'v2'
a._api_version

'v2'

In [32]:
a.grade

3

In [21]:
a.method_2()

In [11]:
a.method_1()

In [12]:
a.grade

1

In [13]:
a.method_2()

In [14]:
a.grade

2

In [5]:
def build_dict():
    return result

big_dict = build_dict()


def func_1():
    global grade  # никогда так не делайте
    
    grade = 1
    print(grade)
    
    return grade
    

def func_2(grade):
    grade += 1

In [7]:
result = func_1()

1


In [8]:
func_2(result)

# Требования к отчету
(в хронологическом порядке)

1. Нужен отчет по трафику (визиты по дням)
2. Сделайте таблицу попроще
3. А можно сразу очень много строк получить?

[Демо-счетчик](https://metrika.yandex.ru/stat/traffic?sort=-ym%3As%3Avisits&period=2020-08-01%3A2020-09-06&accuracy=1&id=44147844&stateHash=5d3ee51607d6bde783131e11) Яндекс.Метрики


In [52]:
import requests

In [53]:
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
METRIKA_ROWS_LIMIT = 5  # max 100000

DEMO_COUNTER = 44147844

In [85]:
class Metrika:
    def __init__(self, token='', counter=DEMO_COUNTER, start_date='2020-09-01', end_date='2020-09-10'):
        self.counter = counter
        self.limit = METRIKA_ROWS_LIMIT
        self.start_date = start_date
        self.end_date = end_date
        
        if token:
            self.headers = {'Authorization': 'OAuth ' + token}
        else:
            self.headers = ''  # для демо-счетчика заголовок запроса должен быть пустым
        
        self.dimensions = ''  # строки отчета
        self.metrics = ''  # столбцы отчета
        self.report = []  # собственно сам отчет
        
    def api_request(self, offset=1):
        """Запрос к API Метрики, возвращает словарь с отчетом"""
        params = {
            'id': self.counter,
            'date1': self.start_date,
            'date2': self.end_date,
            'metrics': self.metrics,
            'dimensions': self.dimensions,
            'limit': self.limit,
            'offset': offset,
            'accuracy': 1,
        }
        
        response = requests.get(API_URL, params=params, headers=self.headers)
        return response.json()['data']
    
    def traffic(self):
        """Отчет по посещаемости"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date'
        
        # report = self.api_request()
        report = self.full_report()
        self.report = report
        
    def browser(self):
        """Отчет по браузерам"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date,ym:s:browser'
        
        # report = self.api_request()
        report = self.full_report()
        self.report = report
    
    def reformat_api_report(self):
        """Упрощение отчета до списка списков"""
        reformatted_report = []

        for line in self.report:
            dimensions = [x['name'] for x in line['dimensions']]
            reformatted_report.append(dimensions + line['metrics'])

        self.report_formatted = reformatted_report
    
    def full_report(self):
        """Постраничная выгрузка из Метрики"""
        full_data = []
        offset = 1
        
        while True:
            print('Starting offset {}'.format(offset))
            data = self.api_request(offset=offset)
            full_data += data
            
            offset += self.limit
            if not data or len(data) < self.limit:
                break
        
        return full_data

In [86]:
m = Metrika(start_date='2022-01-02', end_date='2022-02-15')

In [87]:
m.traffic()

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21
Starting offset 26
Starting offset 31
Starting offset 36
Starting offset 41
Starting offset 46


In [88]:
m.report

[{'dimensions': [{'name': '2022-02-09'}], 'metrics': [706.0]},
 {'dimensions': [{'name': '2022-01-18'}], 'metrics': [642.0]},
 {'dimensions': [{'name': '2022-02-10'}], 'metrics': [637.0]},
 {'dimensions': [{'name': '2022-02-15'}], 'metrics': [620.0]},
 {'dimensions': [{'name': '2022-01-13'}], 'metrics': [617.0]},
 {'dimensions': [{'name': '2022-01-12'}], 'metrics': [594.0]},
 {'dimensions': [{'name': '2022-01-27'}], 'metrics': [591.0]},
 {'dimensions': [{'name': '2022-01-17'}], 'metrics': [590.0]},
 {'dimensions': [{'name': '2022-02-08'}], 'metrics': [589.0]},
 {'dimensions': [{'name': '2022-01-20'}], 'metrics': [588.0]},
 {'dimensions': [{'name': '2022-01-19'}], 'metrics': [579.0]},
 {'dimensions': [{'name': '2022-02-11'}], 'metrics': [576.0]},
 {'dimensions': [{'name': '2022-01-25'}], 'metrics': [573.0]},
 {'dimensions': [{'name': '2022-02-02'}], 'metrics': [567.0]},
 {'dimensions': [{'name': '2022-02-07'}], 'metrics': [565.0]},
 {'dimensions': [{'name': '2022-01-14'}], 'metrics': [5

In [89]:
m.reformat_api_report()

In [90]:
m.report_formatted

[['2022-02-09', 706.0],
 ['2022-01-18', 642.0],
 ['2022-02-10', 637.0],
 ['2022-02-15', 620.0],
 ['2022-01-13', 617.0],
 ['2022-01-12', 594.0],
 ['2022-01-27', 591.0],
 ['2022-01-17', 590.0],
 ['2022-02-08', 589.0],
 ['2022-01-20', 588.0],
 ['2022-01-19', 579.0],
 ['2022-02-11', 576.0],
 ['2022-01-25', 573.0],
 ['2022-02-02', 567.0],
 ['2022-02-07', 565.0],
 ['2022-01-14', 560.0],
 ['2022-02-01', 560.0],
 ['2022-02-14', 544.0],
 ['2022-02-03', 541.0],
 ['2022-01-24', 530.0],
 ['2022-02-04', 527.0],
 ['2022-01-26', 525.0],
 ['2022-01-21', 459.0],
 ['2022-01-10', 442.0],
 ['2022-01-31', 440.0],
 ['2022-01-11', 393.0],
 ['2022-01-04', 380.0],
 ['2022-01-28', 375.0],
 ['2022-01-05', 363.0],
 ['2022-02-12', 362.0],
 ['2022-01-03', 351.0],
 ['2022-01-16', 340.0],
 ['2022-01-06', 339.0],
 ['2022-02-05', 335.0],
 ['2022-01-15', 332.0],
 ['2022-02-06', 324.0],
 ['2022-01-07', 312.0],
 ['2022-02-13', 303.0],
 ['2022-01-29', 302.0],
 ['2022-01-22', 297.0],
 ['2022-01-30', 297.0],
 ['2022-01-23', 

In [91]:
m.browser()

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21
Starting offset 26
Starting offset 31
Starting offset 36
Starting offset 41
Starting offset 46
Starting offset 51
Starting offset 56
Starting offset 61
Starting offset 66
Starting offset 71
Starting offset 76
Starting offset 81
Starting offset 86
Starting offset 91
Starting offset 96
Starting offset 101
Starting offset 106
Starting offset 111
Starting offset 116
Starting offset 121
Starting offset 126
Starting offset 131
Starting offset 136
Starting offset 141
Starting offset 146
Starting offset 151
Starting offset 156
Starting offset 161
Starting offset 166
Starting offset 171
Starting offset 176
Starting offset 181
Starting offset 186
Starting offset 191
Starting offset 196
Starting offset 201
Starting offset 206
Starting offset 211
Starting offset 216
Starting offset 221
Starting offset 226
Starting offset 231
Starting offset 236
Starting offset 241
Starting offset 246
Starting offset 251
St

In [92]:
m.report

[{'dimensions': [{'name': '2022-02-09'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [386.0]},
 {'dimensions': [{'name': '2022-02-10'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [358.0]},
 {'dimensions': [{'name': '2022-02-08'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [344.0]},
 {'dimensions': [{'name': '2022-02-15'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [334.0]},
 {'dimensions': [{'name': '2022-01-18'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [331.0]},
 {'dimensions': [{'name': '2022-01-13'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [326.0]},
 {'dimensions': [{'name': '2022-02-07'},
   {'icon_i

In [93]:
m.reformat_api_report()
m.report_formatted

[['2022-02-09', 'Google Chrome', 386.0],
 ['2022-02-10', 'Google Chrome', 358.0],
 ['2022-02-08', 'Google Chrome', 344.0],
 ['2022-02-15', 'Google Chrome', 334.0],
 ['2022-01-18', 'Google Chrome', 331.0],
 ['2022-01-13', 'Google Chrome', 326.0],
 ['2022-02-07', 'Google Chrome', 321.0],
 ['2022-01-27', 'Google Chrome', 320.0],
 ['2022-02-14', 'Google Chrome', 319.0],
 ['2022-01-25', 'Google Chrome', 316.0],
 ['2022-02-11', 'Google Chrome', 315.0],
 ['2022-01-24', 'Google Chrome', 307.0],
 ['2022-02-01', 'Google Chrome', 306.0],
 ['2022-01-12', 'Google Chrome', 305.0],
 ['2022-01-20', 'Google Chrome', 299.0],
 ['2022-01-17', 'Google Chrome', 293.0],
 ['2022-01-19', 'Google Chrome', 293.0],
 ['2022-01-14', 'Google Chrome', 290.0],
 ['2022-02-02', 'Google Chrome', 285.0],
 ['2022-02-03', 'Google Chrome', 279.0],
 ['2022-01-26', 'Google Chrome', 278.0],
 ['2022-02-04', 'Google Chrome', 273.0],
 ['2022-01-21', 'Google Chrome', 237.0],
 ['2022-01-10', 'Google Chrome', 229.0],
 ['2022-01-31', 